In [26]:
import numpy as np
from numpy.linalg import inv

In [27]:
T = 10
K = 3

In [28]:
Y_train = np.random.random((T, 1))

In [29]:
X_train = np.random.random((T, 3))

## Ridge Regression V1：

Assuming $Y_{label}$ is training label in a form of column: $T \times 1$,
$X_{train}$ is training data in the form of matrix: $T \times K$, where
$K$ is the number of features.

### Step 1: transpose $Y_{train}$

Transpose $Y_{train}$: now $Y_train$ is in the form of $1 \times T$.

In [30]:
Y_train = Y_train.T

In [31]:
X_train[0,:]

array([0.57952266, 0.17686408, 0.70870691])

### Step 2: Create $K$ matrix in the form of $T \times T$

Assuming we declare a penalty $\alpha$

In [32]:
K_matrix = np.zeros((T, T))
alpha = 0.1

for s in range(T):
    for t in range(T):
        K_matrix[s, t] = np.dot(X_train[s,:], X_train[t,:])

In [33]:
additive_term = np.full((T, T), alpha)

K_matrix = K_matrix + additive_term

In [34]:
K_matrix = inv(K_matrix)

### Step 3: Construct k in the form $T \times 1$

In [40]:
x_input = np.random.random((K))

In [41]:
k = np.zeros((T, 1))

In [42]:
for t in range(T):
    k[t] = np.dot(X_train[t, :], x_input)

### Final Step: find $w \cdot x$：

In [46]:
result = np.matmul(np.matmul(Y_train, K_matrix), k)

In [47]:
result

array([[4.]])

As you could the see the result is a number, which means we successfully predict a new
value with training data.

However, this approach is very naive and incomplete. We will have to go back and
calculate the corresponding Lagrange Multiplier and then find the weights explicitly because we
have infinity possibility to compute weights otherwise.

Also, we haven't really implement kernel method. I believe we could replace the dot product
with kernel method. This will extend ridge regression to KRR.

I view the single output as failure because we will need to take advantage of parallelism, which I will
read the spec for np.dot() carefully and make sure our project have the feature.

Lastly, as you see, the inverse is very expensive. We will optimize it.

Otherwise, this V1 is a good place to demonstrate the algorithm which I will keep it.